# Predictability of Total Variation Distance (TVD)

ere we make a comparison of runoff between two locations and ask if the **Total Variation Distance** (TVD) of the distribution of unit area runoff between two locations can be predicted from the attributes of both catchments (and their differences). We will again use the gradient boosted decision tree method.

In [1]:
import os
import pandas as pd
import numpy as np
from time import time

from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook
from bokeh.palettes import Sunset10, Vibrant7

import xgboost as xgb
xgb.config_context(verbosity=2)

from sklearn.metrics import (
    root_mean_squared_error,
    mean_absolute_error,
    roc_auc_score,
    accuracy_score,
)

import data_processing_functions as dpf

from scipy.stats import linregress
output_notebook()

BASE_DIR = os.getcwd()

Loading BokehJS ...

In [2]:
# load the catchment characteristics
fname = 'BCUB_HYSETS_properties_with_climate_with_entropy.csv'
df = pd.read_csv(os.path.join('data', fname))
station_ids = df['official_id'].values
print(f'There are {len(station_ids)} monitored basins in the attribute set.')

There are 1616 monitored basins in the attribute set.


In [3]:
# open an example pairwise results file
pairs_results_folder = os.path.join(
    BASE_DIR, "data", "divergence_test_results",
)
pairs_files = os.listdir(pairs_results_folder)
test_df = pd.read_csv(os.path.join(pairs_results_folder, pairs_files[0]), nrows=1000)
# for f in pairs_files:
#     test_df = pd.read_csv(os.path.join(pairs_results_folder, f))
#     unnamed_cols = [c for c in test_df.columns if c.startswith('Unnamed')]
#     test_df.drop(unnamed_cols, inplace=True, axis=1)
#     test_df.to_csv(os.path.join(pairs_results_folder, f), index=False)
    # print(test_df.columns.tolist())


In [4]:
tvd_columns = [c for c in test_df.columns if 'tvd' in c]
tvd_columns

['tvd_concurrent',
 'tvd_concurrent_max',
 'tvd_nonconcurrent',
 'tvd_nonconcurrent_max']

In [ ]:
terrain = ['drainage_area_km2', 'elevation_m', 'slope_deg', 'gravelius', 'perimeter', 'aspect_deg']
land_cover = [
    'land_use_forest_frac_2010', 'land_use_grass_frac_2010', 'land_use_wetland_frac_2010', 'land_use_water_frac_2010', 
    'land_use_urban_frac_2010', 'land_use_shrubs_frac_2010', 'land_use_crops_frac_2010', 'land_use_snow_ice_frac_2010']
soil = ['logk_ice_x100', 'porosity_x100']
climate = ['prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp', 'high_prcp_freq', 'high_prcp_duration', 'low_prcp_freq', 'low_prcp_duration']
all_attributes = terrain + land_cover + soil + climate
len(all_attributes)

In [7]:
# define the amount of data to set aside for final testing
holdout_pct = 0.10
nfolds = 5
n_boost_rounds = 2500
n_optimization_rounds = 10

#define if testing concurrent or nonconcurrent data
concurrent = 'concurrent'

# the input data file has an associated revision date
revision_date = '20240717'

all_test_results = {}
attribute_set_names = ['climate', '+land_cover', '+terrain', '+soil']

results_folder = os.path.join('./data/', 'tvd_prediction_results')
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

In [8]:
def predict_TVD_from_attributes(target_col, holdout_pct, stations, nfolds, results_folder, partial_counts=False):

    training_stn_cv_sets, test_stn_sets = dpf.train_test_split_by_official_id(holdout_pct, stations, nfolds)
    all_test_results = {}
    for bitrate in [4, 6, 8]:
        t0 = time()
        print(f'bitrate = {bitrate}')
        fname = f"DKL_results_{bitrate}bits_{revision_date}.csv"
        if partial_counts:
            fname = f"DKL_results_{bitrate}bits_{revision_date}_partial_counts.csv"

        fpath = os.path.join(
            BASE_DIR, "data", "divergence_test_results", fname
        )
        df = pd.read_csv(fpath, nrows=None, low_memory=False)
        df.dropna(subset=[target_col], inplace=True)
        t1 = time()
        print(f'    {t1-t0:.2f}s to load input data')
        
        all_test_results[bitrate] = {}
        input_attributes = []

        # add attribute groups successively
        for attribute_set, set_name in zip([land_cover, terrain, soil, climate], attribute_set_names):
            print(f'  Processing {set_name} attribute set.')
            input_attributes += attribute_set 
                        
            features = dpf.format_features(input_attributes)
            
            trial_df, test_df = dpf.run_xgb_trials(
                bitrate, set_name, features, target_col, df, 
                training_stn_cv_sets, test_stn_sets, n_optimization_rounds, 
                nfolds, n_boost_rounds, results_folder
            )
            
            test_rmse = root_mean_squared_error(test_df['actual'], test_df['predicted'])
            test_mae = mean_absolute_error(test_df['actual'], test_df['predicted'])

            print(f'   held-out test rmse: {test_rmse:.2f}, mae: {test_mae:.2f}')
            print('')
            # store the test set predictions and actuals
            all_test_results[bitrate][set_name] = {
                'trials': trial_df, 'test_df': test_df,
                'test_mae': test_mae, 'test_rmse': test_rmse} 
    return all_test_results

In [ ]:
target_col = f'tvd_{concurrent}'

test_results_fname = f'{target_col}_results.npy'
test_results_fpath = os.path.join('data', test_results_fname)
if os.path.exists(test_results_fpath):
    all_test_results = np.load(test_results_fpath, allow_pickle=True).item()
else:
    all_test_results = predict_TVD_from_attributes(target_col, holdout_pct, station_ids, nfolds, results_folder)
    np.save(test_results_fpath, all_test_results)

bitrate = 4
    19.27s to load input data
    0.26 ± 0.004 RMSE mean on the test set (N=10)
  climate
   held-out test rmse: 0.26, mae: 0.21

    0.22 ± 0.007 RMSE mean on the test set (N=10)
  +land_cover
   held-out test rmse: 0.22, mae: 0.16



In [ ]:
plots = []
for b, set_dict in all_test_results.items():
    test_rmse, test_mae = [], []
    attribute_sets = list(set_dict.keys())

    y1 = [set_dict[e]['test_rmse'] for e in attribute_sets]
    y2 = [set_dict[e]['test_mae'] for e in attribute_sets]
    
    source = ColumnDataSource({'x': attribute_sets, 'y1': y1, 'y2': y2})
    
    title = f'{b} bits'
    if len(plots) == 0:
        fig = figure(title=title, x_range=attribute_sets)
    else:
        fig = figure(title=title, x_range=attribute_sets, y_range=plots[0].y_range)
    fig.line('x', 'y1', legend_label='rmse', color='green', source=source, line_width=3)
    fig.line('x', 'y2', legend_label='mae', color='dodgerblue', source=source, line_width=3)
    fig.legend.background_fill_alpha = 0.6
    fig.yaxis.axis_label = 'RMSE'
    
    result_df = pd.DataFrame({'set': attribute_sets, 'rmse': y1, 'mae': y2})
    best_rmse_idx = result_df['rmse'].idxmin()
    best_mae_idx = result_df['mae'].idxmin()
    best_rmse_set = result_df.loc[best_rmse_idx, 'set']
    best_mae_set = result_df.loc[best_mae_idx, 'set']
    best_result = set_dict[best_rmse_set]['test_df']
    
    xx, yy = best_result['actual'], best_result['predicted']
    slope, intercept, r, p, se = linregress(xx, yy)
    
    sfig = figure(title=f'Test: {b} bits best model {best_rmse_set} (N={len(best_result)})')
    sfig.scatter(xx, yy, size=1, alpha=0.6)
    xpred = np.linspace(min(xx), max(xx), 100)
    ybf = [slope * e + intercept for e in xpred]
    sfig.line(xpred, ybf, color='red', line_width=3, line_dash='dashed', legend_label=f'R²={r**2:.2f}')    
    sfig.xaxis.axis_label = 'Actual TVD [bits/sample]'
    sfig.yaxis.axis_label = 'Predicted TVD [bits/sample]'
    sfig.legend.location = 'top_left'
    plots.append(fig)
    plots.append(sfig)

In [ ]:
layout = gridplot(plots, ncols=2, width=350, height=300)
show(layout)

## Citations

```{bibliography}
:filter: docname in docnames
```